In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import os

In [2]:
df = pd.read_csv('NBA_data.csv')
df.head()

,Win Shares,Mins Played (per game),Player,Games,Year,Rank,Points(per game),WS/48,Box +/-,Value Overall Replacement Player,Team,College,Years,Mins Played (Total),Points (Total),Class
0,48.0,30.6,Kenyon Martin,757,2000,1,12.3,0.100,0.7,15.5,NJN,Cincinnati,15,23134,9325,Above Avg
1,0.3,15.6,A.J. Guyton,80,2000,32,5.5,0.010,-5.4,-1.1,CHI,Indiana,3,1246,442,Below Avg
2,11.6,14.3,Jake Voskuhl,450,2000,33,4.0,0.087,-2.2,-0.3,CHI,UConn,9,6443,1814,Below Avg
3,0.6,18.7,Khalid El-Amin,50,2000,34,6.3,0.030,-3.8,-0.4,CHI,UConn,1,936,314,Bust
4,-0.1,10.6,Mike Smith,17,2000,35,3.0,-0.033,-5.5,-0.2,WAS,University of Louisiana at Monroe,1,180,51,Bust


In [33]:
y = df["Class"]
#target_names = ["Player", "Win Shares"]

In [34]:
X = df.drop(["Class", "Player", "Year"], axis=1)
X.head()

,Win Shares,Mins Played (per game),Games,Rank,Points(per game),WS/48,Box +/-,Value Overall Replacement Player,Team,College,Years,Mins Played (Total),Points (Total)
0,48.0,30.6,757,1,12.3,0.100,0.7,15.5,NJN,Cincinnati,15,23134,9325
1,0.3,15.6,80,32,5.5,0.010,-5.4,-1.1,CHI,Indiana,3,1246,442
2,11.6,14.3,450,33,4.0,0.087,-2.2,-0.3,CHI,UConn,9,6443,1814
3,0.6,18.7,50,34,6.3,0.030,-3.8,-0.4,CHI,UConn,1,936,314
4,-0.1,10.6,17,35,3.0,-0.033,-5.5,-0.2,WAS,University of Louisiana at Monroe,1,180,51


In [36]:
#One hot-encoding for Team and College
data = df.values
X = data[:, 0:12]
y = data[:, 8:9]

In [41]:
data

array([[48.0, 30.6, 'Kenyon Martin', ..., 23134, 9325, 'Above Avg'],
       [0.3, 15.6, 'A.J. Guyton', ..., 1246, 442, 'Below Avg'],
       [11.6, 14.3, 'Jake Voskuhl', ..., 6443, 1814, 'Below Avg'],
       ...,
       [5.4, 24.4, 'Landry Shamet', ..., 2679, 1036, 'Above Avg'],
       [2.3, 10.8, 'Robert Williams', ..., 552, 155, 'Below Avg'],
       [0.0, 3.2, 'Kostas Antetokounmpo', ..., 16, 2, 'Below Avg']],
      dtype=object)

In [42]:
X

array([[ 757],
       [  80],
       [ 450],
       [  50],
       [  17],
       [ 717],
       [ 619],
       [  61],
       [ 155],
       [   0],
       [ 629],
       [ 133],
       [ 108],
       [   0],
       [ 341],
       [  19],
       [   3],
       [   6],
       [   0],
       [  51],
       [   0],
       [   0],
       [   0],
       [   0],
       [ 456],
       [  27],
       [ 453],
       [ 547],
       [ 289],
       [1032],
       [ 344],
       [ 436],
       [1326],
       [ 728],
       [ 145],
       [ 409],
       [ 187],
       [ 167],
       [ 592],
       [  69],
       [ 151],
       [ 711],
       [ 205],
       [ 680],
       [ 783],
       [ 643],
       [ 334],
       [   0],
       [ 466],
       [  54],
       [ 878],
       [  22],
       [  31],
       [ 520],
       [ 139],
       [ 115],
       [   0],
       [  39],
       [   0],
       [   0],
       [ 215],
       [  16],
       [  44],
       [  73],
       [   0],
       [ 542],
       [  

In [38]:
# Normalize X show between 1 to -1
X = df["Games"].values.reshape(-1, 1)
y = df["Class"].values.reshape(-1,1)
print("Shape: ", X.shape, y.shape)

Shape:  (863, 1) (863, 1)


In [39]:
#Split into testing and training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [19]:
#Look at other models at this point

In [40]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

C:\Users\newca\.conda\envs\pythondata\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\newca\.conda\envs\pythondata\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")

ValueError: could not convert string to float: 'Chris Taft'